# Capstone Project - The Battle of the Neighborhoods Week 1

## Where to buy a flat for Airbnb in Budapest?

# Applied Data Science Capstone by IBM/Coursera

![alt text](https://www.nationalgeographic.com/content/dam/travel/Guide-Pages/europe/budapest-travel.adapt.1900.1.jpg "Budapest")

## Introduction: Business Problem¶

### Background

Airbnb, the worldwide online marketplace for hospitality services, is getting more and more popular. In the Hungarian capital Budapest - where I currently live - the number of overnight stays reached almost 1.5 million in 2017, 35% more than 2016, according to a 2018 report by Colliers. (Colliers, 2018) In 2017 there were 42.5 thousand Airbnb accommodations in the city, almost as many as in hotels. (Jancsik, Michalkó, Csernyik, 2018)
A lot of investors buy flats specifically to rent them as Airbnb accommodations. The rent for one flat is between 700 and 1000 USD per month. (Jancsik, Michalkó, Csernyik, 2018) However, it is very important that the flat has an excellent location that is attractive for tourists. In Budapest, the Airbnb letting market is mostly concentrated within the downtown area – District V, VI and VII – followed by districts VIII, IX, XIII and I. The top 3 neighbourhoods account for 69% of the total Airbnb supply in Budapest. (Colliers, 2018)

### Problem

Let's imagine that an investor wants to buy a flat to rent it as Airbnb. What is the right location for such a flat? It should be in one of those neighborhoods where there are lots of tourist attractions, bars, restaurants and other venues that are attractive for tourists, because if it is too far away from these places of interest, they will not rent it. However, housing prices should also be considered. In the inner city, average housing prices are much higher, and the investment might not pay off.
The goal of this project is to find the best neighborhoods in Budapest to buy a flat for Airbnb based on the nearby venues of interest and housing prices.

### Interest

The results can be useful for any investor who wants to buy a flat in Budapest to rent as an Airbnb accommodation. The Budapest Airbnb market is driven by professional operators who own multiple accommodations. Around 65% of listings are offered by hosts that have at least two listings, which is much higher than the 40-50% average multi- listers ratio across most other European cities. Around 31% of listings are offered by a host that offers 3-10 accommodations while in the 10+ category it is 19.5%. (Colliers, 2018) However, there are also small investors who rent only one apartment.

## Data

To determine which neighborhoods are the most suitable for Airbnb flats, I will take two main factors into consideration: places of interest in the neighborhood and average housing prices.

First let's import some necessary libraries.

In [3]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    ca-certificates-2019.9.11  |       hecc5488_0         144 KB  conda-forge
    certifi-2019.9.11          |           py36_0         147 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.49-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

### Neighborhood candidates with latitudes and longitudes¶

First, we need the data of the potential neighborhoods. Budapest has 23 districts numbered from I to XXIII. According to the Colliers report, the Airbnb room demand is concentrated within the downtown area and is insignificant in the rest of the districts, so I work only with districts I, V, VI, VII, VIII, IX and XIII. In these 7 districts there are 17 neighborhoods. I got their data from Wikipedia (https://hu.wikipedia.org/wiki/Budapest_v%C3%A1rosr%C3%A9szeinek_list%C3%A1ja), including their coordinates, because the Python Geocoder package was very unreliable. The latitudes and longitudes are all on the separate Wikipedia pages of the neighborhoods, so it could not be scraped, however, I already had the necessary data in csv format, so I used that.

I uploaded the csv file to my Capstone Project in IBM Watson Studio, and here I insert it into a Pandas dataframe.

In [4]:

import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
client_63f6640549c449dbb041cd4650f58c69 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='P1ftm63ObL6BGCYW6JU_F9A-EYCN3zYfzyYa6FqC5mB7',
    ibm_auth_endpoint="https://iam.eu-gb.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.eu-geo.objectstorage.service.networklayer.com')

body = client_63f6640549c449dbb041cd4650f58c69.get_object(Bucket='capstoneproject-donotdelete-pr-ciqgfgwrq5usre',Key='bpneighborhoods.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

latlong = pd.read_csv(body)
latlong


,District,Neighborhood,Latitude,Longitude
0,I.,Tabán,47.491667,19.040833
1,I.,Vár,47.501667,19.033333
2,I.,Gellérthegy,47.486667,19.045556
3,I.,Krisztinaváros,47.496806,19.031933
4,V.,Belváros,47.492397,19.053314
5,V.,Lipótváros,47.502500,19.050833
6,VI.,Terézváros,47.509667,19.069833
7,VII.,Erzsébetváros,47.500556,19.068725
8,VIII.,Corvin-negyed,47.486991,19.073339
9,VIII.,Magdolnanegyed,47.491500,19.084200


Let's explore the neighborhood candidates by creating a map with the neighborhood candidates superimposed on top.

In [5]:
# Using geopy library to get the latitude and longitude values of Budapest.

address = 'Budapest, Hungary'

geolocator = Nominatim(user_agent="budapest_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Budapest are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Budapest are 47.4983815, 19.0404707.


In [6]:
import folium

# create map of Budapest using latitude and longitude values
map_budapest = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, district, neighborhood in zip(latlong['Latitude'], latlong['Longitude'], latlong['District'], latlong['Neighborhood']):
    label = '{}, {}'.format(neighborhood, district)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_budapest)  
    
map_budapest

### Housing prices

The second group of data we need is the average housing price in each potential borough. I found the average price in Hungarian Forint / m2 in each neighborhood on the website https://www.ingatlannet.hu/statisztika/Budapest. Again, the necessary values were on separate pages, so I had to write them manually into a csv file, which I uploaded to my project on IBM Watson Studio and inserted into the code as a Pandas dataframe.

In [7]:
body = client_63f6640549c449dbb041cd4650f58c69.get_object(Bucket='capstoneproject-donotdelete-pr-ciqgfgwrq5usre',Key='bphousing.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

housing = pd.read_csv(body)
housing

,District,Neighborhood,Average price HUF/m2
0,I.,Tabán,970833
1,I.,Vár,916666
2,I.,Gellérthegy,1406771
3,I.,Krisztinaváros,1029796
4,V.,Belváros,2331877
5,V.,Lipótváros,1196756
6,VI.,Terézváros,988124
7,VII.,Erzsébetváros,846815
8,VIII.,Corvin-negyed,695030
9,VIII.,Magdolnanegyed,580193


### Foursquare

Finally, our last data source is the Foursquare API, which can give us the venues of interest in each candidate neighborhood. Let's explore the downtown of Budapest!

Defining Foursquare credentials, version and limit:

In [8]:
CLIENT_ID = 'JB4FCOFLGRUUWKWZ1PCZFHS1J241XYSQMXONTUXSNFYGU4FC'
CLIENT_SECRET = '4TNODEVIX4Z3D20FUL1UGC4H3S24OG053PNI2CI0YXBCZEYQ' 
VERSION = '20180605'
LIMIT = 100

Let's create a function to get the top 100 venues the in each neighborhood within a radius of 500 meters.

In [9]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Run the above function on each neighborhood and create a new dataframe called budapest_venues

In [10]:
budapest_venues = getNearbyVenues(names=latlong['Neighborhood'],
                                   latitudes=latlong['Latitude'],
                                   longitudes=latlong['Longitude']
                                  )

Tabán
Vár
Gellérthegy
Krisztinaváros
Belváros
Lipótváros
Terézváros
Erzsébetváros
Corvin-negyed
Magdolnanegyed
Orczynegyed
Palotanegyed
Tisztviselőtelep
Ferencváros
Angyalföld
Újlipótváros
Vizafogó


In [11]:
budapest_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Tabán,47.491667,19.040833,Oxygen Wellness Naphegy,47.491025,19.037900,Gym / Fitness Center
1,Tabán,47.491667,19.040833,Asztalka,47.492193,19.044231,Dessert Shop
2,Tabán,47.491667,19.040833,Filozófusok kertje,47.489381,19.039051,Sculpture Garden
3,Tabán,47.491667,19.040833,Várkert,47.493569,19.041017,Park
4,Tabán,47.491667,19.040833,Picnic,47.491330,19.044697,Café


Check how many venues were returned for each neighbourhood:

In [12]:
budapest_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Angyalföld,93,93,93,93,93,93
Belváros,100,100,100,100,100,100
Corvin-negyed,100,100,100,100,100,100
Erzsébetváros,100,100,100,100,100,100
Ferencváros,21,21,21,21,21,21
Gellérthegy,18,18,18,18,18,18
Krisztinaváros,42,42,42,42,42,42
Lipótváros,100,100,100,100,100,100
Magdolnanegyed,11,11,11,11,11,11


As we are going to use k-means clustering, it will be useful to use one hot encoding for the venue categories:

In [13]:
# one hot encoding
budapest_onehot = pd.get_dummies(budapest_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
budapest_onehot['Neighborhood'] = budapest_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [budapest_onehot.columns[-1]] + list(budapest_onehot.columns[:-1])
budapest_onehot = budapest_onehot[fixed_columns]

budapest_onehot.head()

,Neighborhood,American Restaurant,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bar,Basketball Court,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Bistro,Board Shop,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Breakfast Spot,Buffet,Building,Burger Joint,Burrito Place,Bus Stop,Café,Camera Store,Carpet Store,Casino,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Cafeteria,Comedy Club,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cruise,Cuban Restaurant,Cultural Center,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant,Electronics Store,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flower Shop,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Gastropub,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Health & Beauty Service,Health Food Store,Hill,Historic Site,History Museum,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hungarian Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indoor Play Area,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Kebab Restaurant,Kids Store,Lake,Laser Tag,Library,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Motorcycle Shop,Mountain,Movie Theater,Multiplex,Music Venue,Nightclub,Noodle House,Office,Optical Shop,Other Great Outdoors,Outdoor Sculpture,Outdoor Supply Store,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Pet Café,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Pub,Radio Station,Record Shop,Restaurant,Road,Rock Club,Roof Deck,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shoe Repair,Shoe Store,Shopping Mall,Smoke Shop,Snack Place,Soccer Stadium,Souvenir Shop,Spa,Speakeasy,Sporting Goods Shop,Steakhouse,Student Center,Supermarket,Sushi Restaurant,Taco Place,Tapas Restaurant,Tattoo Parlor,Tea Room,Thai Restaurant,Theater,Theme Park,Tourist Information Center,Track,Trail,Train Station,Tram Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Water Park,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Tabán,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Tabán,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Tabán,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

Group rows by neighborhood and take the mean of the frequency of occurrence of each category

In [14]:
budapest_grouped = budapest_onehot.groupby('Neighborhood').mean().reset_index()
budapest_grouped

,Neighborhood,American Restaurant,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bar,Basketball Court,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Bistro,Board Shop,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Breakfast Spot,Buffet,Building,Burger Joint,Burrito Place,Bus Stop,Café,Camera Store,Carpet Store,Casino,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Cafeteria,Comedy Club,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cruise,Cuban Restaurant,Cultural Center,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant,Electronics Store,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flower Shop,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Gastropub,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Health & Beauty Service,Health Food Store,Hill,Historic Site,History Museum,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hungarian Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indoor Play Area,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Kebab Restaurant,Kids Store,Lake,Laser Tag,Library,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Motorcycle Shop,Mountain,Movie Theater,Multiplex,Music Venue,Nightclub,Noodle House,Office,Optical Shop,Other Great Outdoors,Outdoor Sculpture,Outdoor Supply Store,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Pet Café,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Pub,Radio Station,Record Shop,Restaurant,Road,Rock Club,Roof Deck,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shoe Repair,Shoe Store,Shopping Mall,Smoke Shop,Snack Place,Soccer Stadium,Souvenir Shop,Spa,Speakeasy,Sporting Goods Shop,Steakhouse,Student Center,Supermarket,Sushi Restaurant,Taco Place,Tapas Restaurant,Tattoo Parlor,Tea Room,Thai Restaurant,Theater,Theme Park,Tourist Information Center,Track,Trail,Train Station,Tram Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Water Park,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Angyalföld,0.010753,0.021505,0.000000,0.000000,0.010753,0.000000,0.010753,0.000000,0.021505,0.032258,0.000000,0.00,0.000000,0.000000,0.000000,0.010753,0.010753,0.00,0.000000,0.000000,0.000000,0.021505,0.000000,0.010753,0.00000,0.021505,0.010753,0.021505,0.032258,0.000000,0.000000,0.00,0.032258,0.000000,0.00,0.010753,0.064516,0.00,0.021505,0.00,0.000000,0.00,0.00,0.000000,0.043011,0.00,0.000000,0.00,0.00,0.00000,0.000000,0.000000,0.000000,0.010753,0.000000,0.021505,0.00,0.000000,0.000000,0.000000,0.000000,0.032258,0.00,0.010753,0.010753,0.00,0.000000,0.000000,0.010753,0.000000,0.000000,0.010753,0.010753,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.00000,0.00,0.000000,0.021505,0.010753,0.010753,0.000000,0.00,0.000000,0.010753,0.000000,0.000000,0.000000,0.000000,0.000000,0.010753,0.00,0.010753,0.00000,0.010753,0.010753,0.010753,0.00000,0.000000,0.00,0.00,0.021505,0.00,0.000000,0.00,0.00,0.010753,0.010753,0.010753,0.000000,0.000000,0.000000,0.010753,0.000000,0.000000,0.000000,0.010753,0.010753,0.000000,0.000000,0.000000,0.00,0.010753,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010753,0.010753,0.000000,0.00,0.000000,0.000000,0.000

Let's see the top 5 most common venues in each neighborhood!

In [15]:
num_top_venues = 5

for hood in budapest_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = budapest_grouped[budapest_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')


----Angyalföld----
               venue  freq
0     Clothing Store  0.06
1     Cosmetics Shop  0.04
2      Women's Store  0.03
3  Electronics Store  0.03
4                Bar  0.03


----Belváros----
                  venue  freq
0           Coffee Shop  0.06
1                 Hotel  0.05
2                  Café  0.05
3  Hungarian Restaurant  0.05
4    Italian Restaurant  0.05


----Corvin-negyed----
            venue  freq
0           Hotel  0.06
1          Bakery  0.04
2  Clothing Store  0.04
3    Dessert Shop  0.04
4     Coffee Shop  0.03


----Erzsébetváros----
                  venue  freq
0           Coffee Shop  0.08
1                 Hotel  0.08
2  Hungarian Restaurant  0.05
3                   Bar  0.05
4            Restaurant  0.05


----Ferencváros----
                  venue  freq
0           Music Venue  0.24
1  Fast Food Restaurant  0.10
2             Jazz Club  0.05
3             Bookstore  0.05
4            Playground  0.05


----Gellérthegy----
               venue  fr

Now let's put this data into a pandas dataframe. Create a function to sort the venues in descending order:

In [16]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Create a new dataframe and display the top 10 venues for each neighborhood:

In [17]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = budapest_grouped['Neighborhood']

for ind in np.arange(budapest_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(budapest_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Angyalföld,Clothing Store,Cosmetics Shop,Electronics Store,Pharmacy,Bar,Women's Store,Chinese Restaurant,Café,Bus Stop,Diner
1,Belváros,Coffee Shop,Café,Hotel,Hungarian Restaurant,Italian Restaurant,Plaza,Eastern European Restaurant,Dessert Shop,Theater,Restaurant
2,Corvin-negyed,Hotel,Dessert Shop,Bakery,Clothing Store,Beer Bar,Coffee Shop,Plaza,Italian Restaurant,Wine Bar,Electronics Store
3,Erzsébetváros,Hotel,Coffee Shop,Hungarian Restaurant,Restaurant,Bar,Burger Joint,Beer Bar,Dessert Shop,Indian Restaurant,Bistro
4,Ferencváros,Music Venue,Fast Food Restaurant,Playground,Gym / Fitness Center,Diner,Cosmetics Shop,Office,Mobile Phone Shop,Café,Fried Chicken Joint
5,Gellérthegy,Playground,Outdoor Sculpture,Grocery Store,Spa,Mountain,Scenic Lookout,Beer Garden,Trail,Dog Run,Eastern European Restaurant
6,Krisztinaváros,Café,Bakery,Plaza,Playground,Grocery Store,Coffee Shop,Restaurant,Bistro,Pizza Place,Chocolate Shop
7,Lipótváros,Italian Restaurant,Restaurant,Coffee Shop,Hotel,Hungarian Restaurant,Ice Cream Shop,Wine Bar,Bistro,Bar,Bakery
8,Magdolnanegyed,Plaza,Bakery,Coffee Shop,Food & Drink Shop,Miscellaneous Shop,Café,Supermarket,Gym / Fitness Center,Burger Joint,Department Store
9,Orczynegyed,Bakery,Park,Flower Shop,Pizza Place,Theme Park,Bus Stop,Food & Drink Shop,Science Museum,Botanical Garden,Lake


We know what the top ten venue categories are in each neighborhood. We can use this information for clustering in the next section.

Now we have all the data we need to find out which Budapest neighborhood wins the battle of the neighborhoods for being the best Airbnb location. To be continued!

## References

Colliers (2018) AIRBNB IN EUROPE -BUDAPEST, https://www.colliers.com/-/media/files/emea/emea/research/hotels/airbnb_budapest_2018_v6.pdf?la=en-gb;  

Ingatlannet.hu (2019) Real estate statistics, https://www.ingatlannet.hu/statisztika/Budapest;  

Jancsik, András and Michalkó, Gábor and Csernyik, Márta (2018) Megosztás megosztottság nélkül – az Airbnb és a budapesti szálláshelypiac átalakulása. KÖZGAZDASÁGI SZEMLE, 65 (3). pp. 259-286. ISSN 0023-4346., http://real.mtak.hu/77930/;  

Wikipedia (2019) List of neighborhoods in Budapest, https://hu.wikipedia.org/wiki/Budapest_városrészeinek_listája  


Cover photo from National Geographic, https://www.nationalgeographic.com/content/dam/travel/Guide-Pages/europe/budapest-travel.adapt.1900.1.jpg